In [69]:
import pandas as pd
from pandas import Timestamp
import time
import os.path
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials

In [18]:
chromedriver_location = "chrome_driver/chromedriver"

In [19]:
# xPaths to each required search element in the field. 
district_drop_down = '//*[@id="courtId"]'
date_field = '//*[@id="inputVO.startDate"]'
#seven_day_search_radio = '//*[@id="sevenDayRadio"]'
search_by_attorney_radio = '//*[@id="searchAttorneyRadio"]'
mobar_input = '//*[@id="inputVO.mobarNumber"]'
submit_button = '//*[@id="findButton"]'

In [20]:
def slice_per(source, step):
    return [source[i::step] for i in range(step)]

In [40]:
main_df = pd.DataFrame({
    "date" : [],
    "case_num" : [],
    "case_style" : [],
    "time": [],
    "days": [],
    "event": [],
    "location": [],
    "judge": []
})

In [46]:
start_date = '8/17/2021' #input('Start Date(MM/DD/YYYY): ')
end_date = '8/24/2021' #input('End Date(MM/DD/YYYY): ')
EMAIL = 'weabooojones93@gmail.com' #input('Enter email address: ')
# get a list of dates in the range specified
datelist = pd.date_range(start_date, end_date).tolist()
# convert the dates to the desired format mm/dd/yyyy
for i in range(len(datelist)):
    datelist[i] = datetime.strftime(datelist[i], '%m/%d/%Y')

In [41]:
# FORM INPUT AND SUBMIT

# create an instance of the webdriver and load the inital calendar search page
driver = webdriver.Chrome(chromedriver_location)
driver.get('https://www.courts.mo.gov/casenet/cases/calendarSearch.do')
# 7 day search results
#driver.find_element_by_xpath(seven_day_search_radio).click()
# search by attorney
driver.find_element_by_xpath(search_by_attorney_radio).click()
# MOBAR number field 
driver.find_element_by_xpath(mobar_input).send_keys("71855")


for date in datelist:
    # clear the date text box
    driver.find_element_by_xpath(date_field).clear()
    # store the district dropdown in an object using Select
    courtDistrictDrp = Select(driver.find_element_by_xpath(district_drop_down))
    courtDistrictDrp.select_by_value("CT11") # 11th District Saint Charles
    # date
    driver.find_element_by_xpath(date_field).send_keys(date)
    # without the pause, the inputs are too fast for casenet to handle
    time.sleep(1) 
    # submit 
    driver.find_element_by_xpath(submit_button).click()

    if len(driver.find_elements_by_xpath('//*[contains(text(), "Your query returned no matches to be viewed at this site.")]')) > 0:
        continue
    # INFO GATHERING
    i=1
    data = []
    raw_text = []
    # get current date
    current_date = driver.find_element_by_xpath('/html/body/table/tbody/tr[2]/td/table/tbody/tr[2]/td/table/tbody/tr[1]/td/table/tbody/tr[1]/td[2]').text
    # collect information from the table
    while driver.find_elements_by_xpath(f'//td[@class="td{i}"]'):
        data.append(driver.find_elements_by_xpath(f'//td[@class="td{i}"]'))
        i += 1
    # parse text into a list
    for d in data:
        for i in d:
            raw_text.append(i.text)       
    # slices the data into lists of each feild 
    organized_data = slice_per(raw_text, 9)
    # create data frame to hold the data
    df = pd.DataFrame({
        "date" : current_date,
        "case_num" : organized_data[0],
        "case_style" : organized_data[1],
        "time": organized_data[2],
        "days": organized_data[3],
        "event": organized_data[5],
        "location": organized_data[7],
        "judge": organized_data[8]
    })
    # add collected data to main dataframe
    main_df = pd.concat([df, main_df], axis=0, join='inner')
    main_df.drop_duplicates(inplace=True)
    main_df.reset_index(drop=True, inplace=True)
    # go back to previous page
    driver.back()

driver.close()


In [42]:

main_df

,date,case_num,case_style,time,days,event,location,judge
0,"Monday, August 23, 2021",1811-CR04181-01,ST V DENNIS ALLEN WRIGHT,8:30 AM,1 OF 1,Probation Viol Hearing,Location: DIVISION 4 COURTROOM St Charles Circ...,Judge/Commissioner: MICHAEL JAMES FAGRAS
1,"Monday, August 23, 2021",1911-CR01773-01,ST V JESSICA MARIE MESHELL,1:30 PM,1 OF 1,Probation Viol Hearing,Location: DIVISION 7 COURTROOM St Charles Circ...,Judge/Commissioner: DANIEL G PELIKAN
2,"Monday, August 23, 2021",1711-CR02341-01,ST V JESSICA MARIE MESHELL,1:30 PM,1 OF 1,Probation Viol Hearing,Location: DIVISION 7 COURTROOM St Charles Circ...,Judge/Commissioner: DANIEL G PELIKAN
3,"Thursday, August 19, 2021",1911-CR01457-01,ST V SEAN D. GWIN,8:30 AM,1 OF 1,Disposition Hearing,Location: DIVISION 4 COURTROOM St Charles Circ...,Judge/Commissioner: MICHAEL JAMES FAGRAS
4,"Thursday, August 19, 2021",1911-CR03589,ST V BRITTANY NICOLE ROSE,1:30 PM,1 OF 1,Preliminary Hearing,Location: DIVISION 10 COURTROOM St Charles Cir...,Judge/Commissioner: NORMAN C STEIMEL
5,"Thursday, August 19, 2021",1911-CR01411,ST V BRITTANY N ROSE,1:30 PM,1 OF 1,Preliminary Hearing,Location: DIVISION 10 COURTROOM St Charles Cir...,Judge/Commissioner: NORMAN C STEIMEL
6,"Thursday, August 19, 2021",2011-CR04156,ST V PERCY LEE BENNETT,1:30 PM,1 OF 1,Disposition Hearing,Location: DIVISION 10 COURTROOM St Charles Cir...,Judge/Commissioner: NORMAN C STEIMEL
7,"Wednesday, August 18, 2021",1611-CR03626-01,ST V NICOLE MARIE MILLER-POLKINGHORNE,9:00 AM,1 OF 1,Bond Appearance Hearing,Location: DIVISION 2 COURTROOM St Charles Circ...,Judge/Commissioner: DEBORAH JEAN ALESSI
8,"Wednesday, August 18, 2021",2011-CR00837,ST V DYLAN T DEPTULA,1:30 PM,1 OF 1,Preliminary Hearing,Location: DIVISION 9 COURTROOM St Charles Circ...,Judge/Commissioner: ELIZABETH WALKER SWANN
9,"Wednesday, August 18, 2021",1911-CR04606,ST V HARRY BERNARD SCHIEN IV,9:00 AM,1 OF 1,Counsel Status Hearing,Location: DIVISION 9 COURTROOM St Charles Circ...,Judge/Commissioner: ELIZABETH WALKER SWANN


In [28]:

# If modifying these scopes, delete the file token.json.
SCOPES = ['https://www.googleapis.com/auth/calendar']

"""Shows basic usage of the Google Calendar API.
Prints the start and name of the next 10 events on the user's calendar.
"""
creds = None
# The file token.json stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first
# time.
if os.path.exists('token.json'):
    creds = Credentials.from_authorized_user_file('token.json', SCOPES)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            'credentials.json', SCOPES)
        creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open('token.json', 'w') as token:
        token.write(creds.to_json())

service = build('calendar', 'v3', credentials=creds)


Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=85388203790-j5s5bfd3ml7cfbaogd98l4ovk0ktrhah.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A64407%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcalendar&state=z7i2TEYggiZKBcY3n7uWUmKMSRMz7O&access_type=offline


In [73]:
time = Timestamp.isoformat(pd.to_datetime(main_df['date'][0] + ' ' + main_df['time'][0]))
event = {
  'summary': 'Google I/O 2015',
  'location': '800 Howard St., San Francisco, CA 94103',
  'description': 'A chance to hear more about Google\'s developer products.',
  'start': {
    'dateTime': time,
    'timeZone': 'America/Chicago',
  },
  'end': {
    'dateTime': time,
    'timeZone': 'America/Chicago',
  },
}

event = service.events().insert(calendarId='weabooojones93@gmail.com', body=event).execute()

In [75]:
for i in range(len(main_df)):
    print(main_df['judge'][i])
    event = {
        'summary': main_df['case_style'][i],
        'description': f'Case #: {main_df["case_num"][i]} \nEvent: {main_df["event"][i]} \n{main_df["judge"][i]} \n{main_df["location"][i]} \nDays: {main_df["days"][i]}',
        'start': {
            'dateTime': Timestamp.isoformat(pd.to_datetime(main_df['date'][i] + ' ' + main_df['time'][i])),
            'timeZone': 'America/Chicago',
        },
        'end': {
            'dateTime': Timestamp.isoformat(pd.to_datetime(main_df['date'][i] + ' ' + main_df['time'][i])),
            'timeZone': 'America/Chicago',
        },
    }

    event = service.events().insert(calendarId=EMAIL, body=event).execute()

Judge/Commissioner: MICHAEL JAMES FAGRAS 
Judge/Commissioner: DANIEL G PELIKAN 
Judge/Commissioner: DANIEL G PELIKAN
Judge/Commissioner: MICHAEL JAMES FAGRAS 
Judge/Commissioner: NORMAN C STEIMEL 
Judge/Commissioner: NORMAN C STEIMEL
Judge/Commissioner: NORMAN C STEIMEL
Judge/Commissioner: DEBORAH JEAN ALESSI 
Judge/Commissioner: ELIZABETH WALKER SWANN 
Judge/Commissioner: ELIZABETH WALKER SWANN


0   2021-08-23 08:30:00
1   2021-08-23 13:30:00
2   2021-08-23 13:30:00
dtype: datetime64[ns]

In [72]:
Timestamp.isoformat(pd.to_datetime(main_df['date'][0] + ' ' + main_df['time'][0]))

'2021-08-23T08:30:00'